# **Object Detection: Using TensorFlow Hub**

#### `Code by @Om Mule`

`Jupyter Notebook` contains code for `Object Detection` using `TensorFlow Hub`

1. Explore `TensorFlow Hub` for models
2. Import model into workspace
3. Preprocess image for getting `predictions`.
4. Feed the preprocessed images to model to get `predictions`.

## Import Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns

import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
%matplotlib inline

`Tensorflow Hub` is a repository of trained `Machine Learning models` which you can reuse in your own projects. 


Link for `TensorFlow Hub`:  [TensorFlow Hub](https://www.tensorflow.org/hub)

Link for `Model`:   [Model URL](https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1) 



In [2]:
# Importing the model from TensorFlow Hub

module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

### Load the model

We will load the `model` using the `module handle` specified

In [3]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


### Choosing the `signatures`

Some models in the Tensorflow hub can be used for different tasks. So each model's documentation should show what `signature` to use when running the model.

In [ ]:
# Get all signatures

model.signatures.keys()

Please choose the `'default'` signature for your `object detector.`
- For `object detection` models, its 'default' signature will accept a batch of `image tensors` and output a `dictionary` describing the objects detected, which is what you'll want here.

In [6]:
detector = model.signatures['default']

### Download & Resize image

This function `downloads` an image from a `URL` & `preprocesses` it & then `saves` it to the disk.

-`tempfile package` in Python is used to create a temporary file of the `image.`

In [7]:
def download_and_resize_image(url, new_width=256, new_height=256):
  # Fetch an image online, resizes & saves it locally

  # Create a JPG file instance
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()

  # Store the image data in memory buffer
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)

  # Resize the image using Pillow (if aspect ratio is different)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")

  # Save image to temporary .jpg file created earlier
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s."% filename)
  return filename

In [8]:
# Download and preprocess the image
url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# Using original height & width
downloaded_image_path = download_and_resize_image(url, 3872, 2592)

Image downloaded to /tmp/tmpxdrb73ju.jpg.


## Run the detector

Run the detector with `preprocessing functions` & using the `model` 

In [10]:
def load_img(path):
  # Load a JPEG image & convert it to a tensor
  img = tf.io.read_file(path)

  # Conversion to a tensor
  img = tf.image.decode_jpeg(img, channels=3)
  return img


def run_detector(detector, path):
  img = load_img(path)
  # Add a batch dimension in front of tensor
  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  # Get Inference
  result = detector(converted_img)
  # Save results
  result = {key:value.numpy() for key,value in result.items()}
  # print results
  print("Found %d objects." % len(result["detection_scores"]))

  print(result["detection_scores"])
  print(result["detection_class_entities"])
  print(result["detection_boxes"])

## Run the detector

In [11]:
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.6532174  0.61050487 0.60152495 0.59255666 0.5917778  0.58154964
 0.550532   0.4957557  0.47424877 0.47322133 0.44066477 0.4051141
 0.39803803 0.3940655  0.3714858  0.36155996 0.3615076  0.34689182
 0.33362356 0.31252867 0.2887799  0.25758353 0.25748897 0.25196075
 0.24782148 0.23412406 0.20432054 0.20324662 0.17988308 0.17964761
 0.1737451  0.16431497 0.16031103 0.15895295 0.15620388 0.15468779
 0.14754449 0.13622597 0.12740111 0.12555613 0.12102704 0.11813033
 0.11387308 0.11229108 0.11129158 0.09718708 0.09137249 0.08976003
 0.0888024  0.08633561 0.08337442 0.08095147 0.07988831 0.0774143
 0.0773216  0.07631025 0.07507852 0.07386057 0.07233316 0.07204045
 0.07110182 0.06935571 0.06825587 0.06427855 0.0624855  0.06226387
 0.06211156 0.05940022 0.05798778 0.05784462 0.05725535 0.05346682
 0.05304348 0.05242869 0.04892596 0.04813037 0.04578473 0.04424176
 0.04337559 0.04277798 0.04262636 0.04163509 0.04081756 0.03976011
 0.03945961 0.03944291 0.03863972 0.03769058 